<a href="https://colab.research.google.com/github/harry-graves/Aria_ORI/blob/main/aria_calibration_parameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Running from Google Colab, installing projectaria_tools")
!pip install projectaria-tools

In [ ]:
from projectaria_tools.core import data_provider, calibration
from projectaria_tools.core.image import InterpolationMethod
from projectaria_tools.core.sensor_data import TimeDomain, TimeQueryOptions
from projectaria_tools.core.stream_id import RecordableTypeId, StreamId
import numpy as np
import os
from matplotlib import pyplot as plt
from google.colab import drive

In [ ]:
# Upload your .vrs file and change the following filename
vrsfile = 'ET1.vrs'

In [ ]:
print(f"Creating data provider from {vrsfile}")
provider = data_provider.create_vrs_data_provider(vrsfile)
if not provider:
    print("Invalid vrs data provider")

In [ ]:
device_calib = provider.get_device_calibration()
all_sensor_labels = device_calib.get_all_labels()
print(f"device calibration contains calibrations for the following sensors \n {all_sensor_labels}")

In [ ]:
# Retrieve device calibration
device_calib = provider.get_device_calibration()

# Define the sensors to retrieve data for
sensors = ["imu-left", "imu-right", "camera-slam-left", "camera-slam-right", "camera-rgb"]

# Iterate over each sensor and print the required information
for sensor in sensors:
    if "imu" in sensor:
        # Retrieve and print IMU extrinsics
        imu_calib = device_calib.get_imu_calib(sensor)
        if imu_calib is not None:
            print(f"{imu_calib.get_label()} has extrinsics transform_Device_Imu:\n{imu_calib.get_transform_device_imu().to_matrix()}\n")
    else:
        # Retrieve and print camera extrinsics and intrinsics
        camera_calib = device_calib.get_camera_calib(sensor)
        if camera_calib is not None:
            extrinsics = device_calib.get_transform_device_sensor(sensor).to_matrix()
            focal_lengths = camera_calib.get_focal_lengths()
            image_size = camera_calib.get_image_size()
            principal_point = camera_calib.get_principal_point()
            print(f"{sensor} has extrinsics of \n{extrinsics}")
            print(f"{sensor} has focal lengths: {focal_lengths}")
            print(f"{sensor} has principal point: {principal_point}")
            print(f"{sensor} has image size: {image_size}\n")

# Not sure there is a function for the distortion coefficients :(

In [ ]:
# Put everything in the right IMU frame

device_name = "imu-right"
imu_calib = device_calib.get_imu_calib(device_name)
print(f"Device calibration origin label {device_calib.get_origin_label()}")
T_Device_Imu = imu_calib.get_transform_device_imu().to_matrix()
print(f"{imu_calib.get_label()} has extrinsics T_Device_Imu:\n {T_Device_Imu} \n")

device_name = "camera-slam-left"
cam_calib = device_calib.get_camera_calib(device_name)
print(f"Device calibration origin label {device_calib.get_origin_label()}")
T_Device_Device = device_calib.get_transform_device_sensor(device_name).to_matrix()
print(f"{cam_calib.get_label()} has extrinsics T_Device_Device:\n {T_Device_Device} \n")

device_name = "camera-slam-right"
cam_calib = device_calib.get_camera_calib(device_name)
print(f"Device calibration origin label {device_calib.get_origin_label()}")
T_Device_Cam = device_calib.get_transform_device_sensor(device_name).to_matrix()
print(f"{cam_calib.get_label()} has extrinsics T_Device_Cam:\n {T_Device_Cam} \n")

T_Cam_Device = np.linalg.inv(T_Device_Cam)
T_Cam_Imu = T_Cam_Device @ T_Device_Imu
print(f"{cam_calib.get_label()} has extrinsics T_Cam_Imu: \n {T_Cam_Imu} \n")